In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras.engine.topology import Layer
from keras.datasets import mnist
import pdb

Using TensorFlow backend.


In [2]:
batchsize = 64
n_z = 20
m = 64

In [3]:
def encoder(inputs):

    encode = Conv2D(32, (2, 2), strides=2)(inputs)

    encode = MaxPooling2D((2, 2))(encode)

    encode = Activation('sigmoid')(encode)
    
    out_shape = list(k.int_shape(encode))
    #print(out_shape)

    encode = Conv2D(32, (2, 2), strides=2)(encode)

    encode = MaxPooling2D((2, 2))(encode)

    encode = Activation('sigmoid')(encode)
    
    out_shape = list(k.int_shape(encode))
    #print(out_shape)

    encode = Flatten()(encode)

    encode = Dense(512, activation = 'relu')(encode)
    
    encode = Dropout(0.5)(encode)

    mu = Dense(n_z, activation = 'linear')(encode)

    log_sigma = Dense(n_z, activation = 'linear')(encode)
    
    return mu, log_sigma

In [4]:
def decoder(input2): 
    
    decode = Dense(shape[1] * shape[2] * shape[3], activation = 'relu')(latent_vector)
    
    out_shape = list(k.int_shape(decode))
    #print(out_shape)
    decode = Reshape((shape[1], shape[2], shape[3]))(decode)
    
    decode = Conv2D(32, (2, 2), strides=2)(decode)
    
    decode = UpSampling2D((2, 2))(decode)
    
    decode = Activation('relu')(decode)
    
    decode = Conv2D(32, (2, 2), strides=2)(decode)
    
    decode = UpSampling2D((2, 2))(decode)
    
    decode = Activation('relu')(decode)
    
    out = Conv2D(1, (2, 2), activation = 'sigmoid')(decode)
    
    mu_out = Dense(n_z)(out)

    log_sigma_out = Dense(n_z)(out)
    
    return mu_out, log_sigma_out, out

In [ ]:
def loss():
    # p_z = gaussian(0,1)
    # q_z_x = gaussian(mean_y, e^ln_var_y) - check this exp
    # p_x_z = gaissian(mean_x, e^ln_var_x)
    # L = F.gaussian_kl_dvergance(q_z_x, p_z) - F.gaussian_nll(p_x_z)
        
    #? What about input x?
        
    # Why use log variance instead of variance:

In [5]:
def sample_z(args):
    mu, log_sigma = args
    batch = K.shape(mu)[0]
    dim = K.int_shape(mu)[1]
    eps = K.random_normal(shape=(batch, dim), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
batches = int(len(x_train)/batchsize)

In [ ]:
type(x_train)

In [ ]:
x_train.shape

In [ ]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            x = []
            
            data = x_train[batch * batchsize: (batch + 1) * batchsize]
            for i in data:
                x.append(np.reshape(i, (28, 28, 1)))
            
            x_train1 = np.asarray(x)
            yield (x_train1, x_train1) # the data is also the target (since we're re-generating images)

In [7]:
inputs1 = Input(shape = (28, 28, 1))

mu, log_sigma = encoder(inputs1)

#pdb.set_trace()

latent_vector = Lambda(sample_z, output_shape=(n_z,))([mu, log_sigma])

model_encoder = Model(inputs = inputs1, outputs = [mu, log_sigma, latent_vector])

inputs2 = Input(shape = (n_z,))

mu_out, log_sigma_out, out_img = decoder(inputs2)

'latent_vector = sample_z(mu, log_sigma)\n\nmu_out, log_sigma_out, out_img = decoder(latent_vector)\n'

In [8]:
model_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 32)   160         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 7, 7, 32)     0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 7, 7, 32)     0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
model.fit_generator(batch_generator(), steps_per_epoch = batches, epochs=10, shuffle = True, callbacks = callback_list)